In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5076126406940172214,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8776031448444269900
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 4985044352
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1970875123069243611
 physical_device_desc: "device: 0, name: GeForce GTX 1660 SUPER, pci bus id: 0000:06:00.0, compute capability: 7.5",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 397581059622402012
 physical_device_desc: "device: XLA_GPU device"]

In [2]:
tf.__version__

'2.2.0'

In [3]:
tf.debugging.set_log_device_placement(True)

In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint
import os
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from numpy import expand_dims
from keras import models
from keras import layers
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.python.keras.layers import Dense, Dropout, Input
from tensorflow.python.keras.models import Model
from tensorflow.keras.layers import Input, Dense

model = models.Sequential()
#1층
model.add(Conv2D(input_shape=(224,224,3), filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#2층
model.add(Conv2D(input_shape=(224,224,64),filters=64, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#Max Pooling
model.add(MaxPooling2D(pool_size =(2,2), strides=(2,2)))
#3층
model.add(Conv2D(input_shape=(112,112,64),filters=128, kernel_size=(3,3), strides=(1,1), padding ='same', activation='relu'))
#4층
model.add(Conv2D(input_shape=(112,112,128),filters=128, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#Max Pooling
model.add(MaxPooling2D(pool_size =(2,2), strides=(2,2)))
#5층
model.add(Conv2D(input_shape=(56,56,128), filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#6층
model.add(Conv2D(input_shape=(56,56,256), filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#7층
model.add(Conv2D(input_shape=(56,56,256), filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
#8층
model.add(Conv2D(input_shape=(28,28,256), filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#9층
model.add(Conv2D(input_shape=(28,28,512), filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#10층
model.add(Conv2D(input_shape=(28,28,512), filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
#11층
model.add(Conv2D(input_shape=(14,14,512), filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#12층
model.add(Conv2D(input_shape=(14,14,512), filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#13층
model.add(Conv2D(input_shape=(14,14,512), filters=512, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
#Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
#14층 (fc1)
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
#15층 (fc2)
model.add(Dense(4096,activation='relu'))
model.add(Dense(2048,activation='relu'))
model.add(Dense(1024,activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(2,activation='softmax'))

In [12]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 56, 56, 256)      

In [5]:
###vgg16으로 custom data training해보기###
# cats and dogs dataset 활용

In [20]:
#labelimg로 xray상의 alcohol에 라벨링
#train = 125개 ,test = 335개
train_dir = 'C:/Users/user/Desktop/catsanddogs/training_set/training_set'
test_dir = 'C:/Users/user/Desktop/catsanddogs/test_set/test_set'

# 폴더에 따라 자동 분류
train_image_generator = ImageDataGenerator(rescale=1./255)
test_image_generator = ImageDataGenerator(rescale=1./255)

# 데이터 구조 생성
train_data_gen = train_image_generator.flow_from_directory(batch_size=16,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(224, 224),
                                                           class_mode='binary')

test_data_gen = test_image_generator.flow_from_directory(batch_size=16,
                                                         directory=test_dir,
                                                         target_size=(224, 224),
                                                         class_mode='binary')

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [21]:
model.compile(optimizer='adam',
             loss = 'binary_crossentropy',
             metrics=['accuracy'])

In [ ]:
history = model.fit_generator(
    train_data_gen, epochs=25,
    validation_data=test_data_gen,
    )
model.save('trained_vgg16.h5')

# 최종 결과 리포트
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

from matplotlib import pyplot as plt

plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='testing acc')
plt.title('Training and testing accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='testing loss')
plt.title('Training and testing loss')
plt.legend()

plt.show()

# 저장 모델 불러오기
from keras.models import load_model

new_model = load_model("newVGG16.h5")

Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Epoch 1/25
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing 

Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Epoch 2/25
501/501 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.4973Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
501/501 [==============================] - 135s 269ms/step - loss: 0.6931 - accuracy: 0.4973 - val_loss: 0.6931 - val_accuracy: 0.4998
Executing op OptimizeDatas